In [228]:
# Read in data
store_data = pd.read_csv('manhattandata.csv')
data = store_df.iloc[:, np.r_[0:9, 11:19, 23:35].copy()
transit_df = pd.read_csv('transit_data.csv')

In [ ]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.features import DivIcon
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve,  f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [ ]:
distance_list = []
for i,row in data.iterrows():
    a = row.latitude, row.longitude
    distances = []
    for j,row2 in transit_df.iterrows():
        b = row2.station_lat, row2.station_long
        distances.append(geopy.distance.geodesic(a, b).m)
        min_distance = min(distances)
        distance_list.append(min_distance)

data['minsubdist'] = distance_list
data.head()

In [ ]:
# Duplicate rows in subway data to reflect the number of transit lines at a station (not including commuter rail):
transit_lines = transit_df.reindex(transit_df.index.repeat(transit_df.num_lines))

In [ ]:
distance8 = []
for i,row in data.iterrows():
    a = row.latitude, row.longitude
    distances = []
    for j,row2 in transit_lines.iterrows():
        b = row2.station_lat, row2.station_long
        distances.append(geopy.distance.geodesic(a, b).m)
    distances.sort()
    eightclosest = distances[0]+distances[1]+distances[2]+distances[3]+distances[4]+distances[5]+distances[6]+distances[7]
    distance8.append(eightclosest)
    
data['sub8closest'] = distance8

In [ ]:
data['logfloors'] = np.log(data['numfloors'])
data['loglot'] = np.log(data['lotarea'])
data['logcomarea'] = np.log(data['comarea'])
data['logassesstot'] = np.log(data['assesstot'])
data['logresunits'] = np.log(data['unitsres']+1)

In [ ]:
sns.histplot(data=fulldata, x="logfloors", bins=30, color="purple").set(title='Number of Floors in Building (log-transformed)')

In [ ]:
sns.histplot(data=fulldata, x="loglot", bins=50, color="red").set(title='Building Lot Size (log-transformed)')

In [ ]:
sns.histplot(data=fulldata, x="logassesstot", bins=50, color="green").set(title='Total Assessed Value of Building & Land (log-transformed)')

In [ ]:
sns.histplot(data=fulldata, x="logcomarea", bins=50, color="darkblue").set(title='Commercial Square Footage in Building (log-transformed)')

In [ ]:
sns.histplot(data=fulldata, x="logresunits", bins=50).set(title='Residential Units in Building (log-transformed)')

In [ ]:
data['built'] = 2019 - data['yearbuilt']
data['altered'] = data['yearalter1'].apply(lambda x: 0 if x == 0 else 1)
data['vacant01'] = data['vacant'].apply(lambda x: 0 if x == 'NO' else 1)

In [ ]:
fig, ax = plt.subplots()

def label_function(val):
    return f'{val / 100 * len(data):.0f}\n{val:.0f}%'

data.groupby('vacant').size().plot(kind='pie', autopct=label_function, startangle=370,
                                   wedgeprops = {"edgecolor" : "black", 'linewidth': 0.6, 'antialiased': True}, 
                                   textprops={'fontsize': 16}, colors=['cornflowerblue', 'orangered'], figsize=(6,6))
ax.set_ylabel('')
ax.set_xlabel('Storefronts in Manhattan:\nVacant or Not', size=18)
plt.show

In [ ]:
vacancies = pd.DataFrame()
vacancies = data[data['vacant01'] == 1].copy()

manhattan = vacancies[['latitude', 'longitude']]
manhattan_map = folium.Map(location=[manhattan.latitude.mean()+.03, manhattan.longitude.mean()+.02], zoom_start=11,
                      control_scale=True)
vacancies['latitude'] = vacancies['latitude'].astype(float)
vacancies['longitude'] = vacancies['longitude'].astype(float)
heat_df = vacancies[['latitude', 'longitude']]
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]
HeatMap(heat_data, min_opacity=.4, radius=7, blur=6).add_to(manhattan_map)
manhattan_map

In [ ]:
# Generate map:
manhattan = vacancies[['latitude', 'longitude']]
NTA_map2 = folium.Map(location=[manhattan.latitude.mean()+.03, manhattan.longitude.mean()+.02], zoom_start=11,
                      control_scale=True)
folium.GeoJson('ManhattanNoPark.geojson', tooltip=folium.features.GeoJsonTooltip(
                        fields=['ntaname'], alias=['Neighborhood'], localize=True, sticky=False, labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """, max_width=4000,),).add_to(NTA_map2)

# Label neighborhoods in Lower Manhattan:
circle_lat1 = 40.707779484290015
circle_lon1 = -74.01169125742088
folium.map.Marker(
    [circle_lat1+.001, circle_lon1-.00055],
    icon=DivIcon(icon_size=(250,72), icon_anchor=(0,0), html='<div style="font-size: 15pt">1</div>',)
    ).add_to(NTA_map2)

circle_lat2 = 40.72150647250581
circle_lon2 = -74.00654137332417
folium.map.Marker(
    [circle_lat2+.001, circle_lon2-.00055],
    icon=DivIcon(icon_size=(250,72), icon_anchor=(0,0), html='<div style="font-size: 15pt">2</div>',)
    ).add_to(NTA_map2)

circle_lat3 = 40.71496860850175
circle_lon3 = -73.99538338315665
folium.map.Marker(
    [circle_lat3+.001, circle_lon3-.00055],
    icon=DivIcon(icon_size=(250,72), icon_anchor=(0,0), html='<div style="font-size: 15pt">3</div>',)
    ).add_to(NTA_map2)

circle_lat4 = 40.71639983755882
circle_lon4 = -73.98491203868387
folium.map.Marker(
    [circle_lat4+.001, circle_lon4-.00055],
    icon=DivIcon(icon_size=(250,72), icon_anchor=(0,0), html='<div style="font-size: 15pt">4</div>',)
    ).add_to(NTA_map2)

folium.Circle([circle_lat1, circle_lon1], 100, fill=True, fill_opacity=100, fill_color='#eaebed').add_to(NTA_map2)
folium.Circle([circle_lat2, circle_lon2], 100, fill=True, fill_opacity=100, fill_color='#eaebed').add_to(NTA_map2)
folium.Circle([circle_lat3, circle_lon3], 100, fill=True, fill_opacity=100, fill_color='#eaebed').add_to(NTA_map2)
folium.Circle([circle_lat4, circle_lon4], 100, fill=True, fill_opacity=100, fill_color='#eaebed').add_to(NTA_map2)

# Overlay heat map:
vacancies['latitude'] = vacancies['latitude'].astype(float)
vacancies['longitude'] = vacancies['longitude'].astype(float)
heat_df = vacancies[['latitude', 'longitude']]
heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]
HeatMap(heat_data, min_opacity=0.9, radius=7, blur=5).add_to(NTA_map2)
NTA_map2

In [132]:
dfg = pd.DataFrame(data.groupby(['neighborhood'])['vacant01'].mean()*100)
dfgsort = dfg.sort_values(by='vacant01', ascending=False)
dfgsort.plot(kind='barh', title='Vacancy Percentages by Neighborhood', xlabel='Neighborhood', figsize=(9,8), legend=None)

In [134]:
x = data.loc[:,['latitude','loglot', 'logfloors', 'logcomarea','logassesstot',
             'logresunits', 'built', 'Pop_1E', 'edbachpct', 'edgradpct', 'edlthsaggpct', 'dfhspct',
             'fbpct', 'RntVacRtE', 'mv10ltrpct', 'mv00t09pct', 'mv90t99pct', 'mvbf89pct', 'MdGRE',
              'minsubdist', 'altered', 'sub8closest']]

In [135]:
y = data['vacant01']

In [136]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.30, random_state=8888)

In [137]:
xnumeric_train = X_train.loc[:,['latitude','loglot','logfloors', 'logcomarea','logassesstot',
             'logresunits', 'built', 'Pop_1E', 'edbachpct', 'edgradpct', 'edlthsaggpct', 'dfhspct',
             'fbpct', 'RntVacRtE', 'mv10ltrpct', 'mv00t09pct', 'mv90t99pct', 'mvbf89pct', 'MdGRE',
              'minsubdist', 'sub8closest']]

In [138]:
X_train_norm = pd.DataFrame()
X_train_norm = xnumeric_train.apply(lambda x : x/(2*(x.std())))
X_train_norm['altered'] = X_train['altered']
X_train_norm

In [139]:
xnumeric_test = X_test.loc[:,['latitude','loglot','logfloors', 'logcomarea','logassesstot',
             'logresunits', 'built', 'Pop_1E', 'edbachpct', 'edgradpct', 'edlthsaggpct', 'dfhspct',
             'fbpct', 'RntVacRtE', 'mv10ltrpct', 'mv00t09pct', 'mv90t99pct', 'mvbf89pct', 'MdGRE',
              'minsubdist', 'sub8closest']]

In [140]:
X_test_norm = pd.DataFrame()
X_test_norm = xnumeric_test.apply(lambda x : x/(2*(x.std())))
X_test_norm['altered'] = X_test['altered']
X_test_norm

In [241]:
# Train logistic regression model:

lr = LogisticRegression()
lr = LogisticRegression(max_iter=10000).fit(X_train_norm, y_train)
print(lr.coef_, lr.intercept_)

In [ ]:
y_pred = lr.predict(X_test_norm)
print(classification_report(y_test, y_pred))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred), 2))

In [244]:
def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= confusion_matrix(y, pred, 
            normalize='true')
    return pd.DataFrame([[f'TN = {tn} (TNR = {tnr:1.2%})', 
                                f'FP = {fp} (FPR = {fpr:1.2%})'], 
                         [f'FN = {fn} (FNR = {fnr:1.2%})', 
                                f'TP = {tp} (TPR = {tpr:1.2%})']],
            index=['True 0(Full)', 'True 1(Vacant)'], 
            columns=['Pred 0(Predicted Full)', 
                            'Pred 1(Predicted Vacant)'])

In [ ]:
conf_matrix(y_test, y_pred)

In [ ]:
predtst=lr.predict_proba(X_test_norm)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, predtst)
dfplot=pd.DataFrame({'Threshold':thresholds, 
        'False Positive Rate':fpr, 
        'False Negative Rate': 1.-tpr})
ax=dfplot.plot(x='Threshold', y=['False Positive Rate',
        'False Negative Rate'], figsize=(10,6))
ax.set_xbound(0,1); ax.set_ybound(0,1)

In [ ]:
# Tune classification threshold:

tuned_thresh = np.where(predtst >= 0.092, 1, 0)
conf_matrix(y_test, tuned_thresh)

In [ ]:
# Print classification report:
print(classification_report(y_test, tuned_thresh))
print('Accuracy score: ', round(accuracy_score(y_test, tuned_thresh), 2))
print('F1 Score: ', round(f1_score(y_test, tuned_thresh), 3))

In [ ]:
# Generate confusion matrix:
cm = confusion_matrix(y_test, tuned_thresh)
plt.figure(figsize=(9,9))
sns.set(font_scale=2)
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Confusion Matrix', size = 15);

In [ ]:
# Estimate random forest and iterate over various numbers of trees:

RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)

oob_list = list()

for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400, 500, 1000]:
    print(n_trees)
    RF.set_params(n_estimators=n_trees)
    RF.fit(X_train_norm, y_train)
    oob_error = 1 - RF.oob_score_
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

rf_oob_df

In [ ]:
%matplotlib inline

sns.set_context('talk')
sns.set_style('white')
ax = rf_oob_df.plot(legend=False, marker='o', figsize=(14,7), linewidth=5)
ax.set(ylabel='out-of-bag error');

In [187]:
# Random forest with 300 trees:
RC = RandomForestClassifier(n_estimators=300, random_state=42)
RC = RC.fit(X_train_norm, y_train)
y_predict = RC.predict(X_test_norm)

In [ ]:
print(classification_report(y_test, y_predict))
print('Accuracy score: ', round(accuracy_score(y_test, y_predict), 2))
print('F1 Score: ', round(f1_score(y_test, y_predict), 2))

In [208]:
# Tune the threshold (examine changes in F1 score and other metrics above
# with different thresholds)
threshold = 0.277
predicted_proba = RC.predict_proba(X_test_norm)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

In [ ]:
# Print classification report:
print(classification_report(y_test, predicted))
print('Accuracy score: ', round(accuracy_score(y_test, predicted), 2))
print('F1 Score: ', round(f1_score(y_test, predicted), 4))

In [ ]:
cm = confusion_matrix(y_test,predicted)
plt.figure(figsize=(9,9))
sns.set(font_scale=2)
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Confusion Matrix', size = 15);

In [ ]:
# Plot feature importances:

importances = RC.feature_importances_
implist = importances.tolist()
bardata = pd.DataFrame()
columns = ['Latitude', 'Lot Area', 'Number of Floors', 'Commercial Area of Building', 'Total Assessed Building & Land Value', 
           'Number of Residential Units', 'Age of Building', 'Neighborhood Population', 'Highest Degree: Bachelors', 
           'Highest Degree: Graduate', 'Highest Degree: Less Than Bachelors', '% Lived in Different House Last Year', 
           'Foreign-Born %', 'Rental Vacancy Rate', '% Moved To Home in 2010 or Later', '% Moved To Home 2000-2009', 
           '% Moved To Home 1990-1999', '% Moved To Home 1989 or Earlier', 'Median Gross Rent', 'Minimum Distance to Rail', 
           'Sum of Min 8 Distances to Rail', 'Building Ever Altered']

bardata['cols'] = columns
bardata['Importances'] = implist
barsort = bardata.sort_values(by='Importances')
plt.figure(figsize=(8,10))
plt.title('Feature Importances from Random Forest Model', size=30)
plt.barh(barsort.cols, barsort.Importances, color='limegreen')

In [ ]:
# Examine correlations with key features:

ycorr = data['vacant01']
correlation = ycorr.corr(data['minsubdist'])
correlation

In [ ]:
ycorr = data['vacant01']
correlation = ycorr.corr(data['sub8closest'])
correlation

In [ ]:
ycorr = data['vacant01']
correlation = ycorr.corr(data['logassesstot'])
correlation

In [ ]:
ycorr = data['vacant01']
correlation = ycorr.corr(data['latitude'])
correlation

In [ ]:
ycorr = data['vacant01']
correlation = ycorr.corr(data['logcomarea'])
correlation

In [ ]:
ycorr = data['vacant01']
correlation = ycorr.corr(data['loglot'])
correlation